In [89]:
from crossref.restful import Works
works = Works()
pyear = '2018'
number = works.filter(has_funder='true').filter(has_award='true').filter(from_pub_date=pyear).filter(until_pub_date=pyear).count()
print(number)

740961


In [91]:
counter =705500
print("{:09d}".format(counter), "something:", counter, 'name_'+pyear+'.csv')

print("Advance: {:3.6f}%".format(counter/number*100))

000705500 something: 705500 name_2018.csv
Advance: 95.214188%


In [104]:
from crossref.restful import Works

def award_in_crossref(aw):
    ukch_wks =[]

    for wk in aw:      
        awd_list = [] 
        for fdr in wk['funder']:
            if 'award' in fdr.keys():
               awds = 0
               for awd in fdr['award']:
                   if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                              'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                       awd_list.append(awd)
        if len(awd_list) > 0:
            ukch_wks.append(wk)
    return ukch_wks

def affi_in_crossref(aw):
    ukch_wks = []
    for wk in aw:
        ukch_affiliation = False
        if 'author' in wk.keys() :
            for autr in wk['author']:
                if 'affiliation' in autr.keys():
                    for affi in autr['affiliation']:
                        if "UK Catalysis Hub" in affi['name']:
                            ukch_affiliation = True
                            break
                    if ukch_affiliation:
                        ukch_wks.append(wk)
                        break
                        
    return  ukch_wks


In [51]:
foud_pubs = {}
for wk in ukch_grant:
    art_authors = ""
    if 'author' in wk.keys() :
        for autr in wk['author']:
            if art_authors == "":
                art_authors = autr['family']+", " + autr ['given']
            else:
                art_authors += ", " + autr['family']+", "+ autr ['given']
    fund_award = ""
    for fdr in wk['funder']:
        if 'award' in fdr.keys():
          for awd in fdr['award']:
               if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                          'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                    if fund_award  == "":
                        fund_award = awd
                    else:
                        fund_award += ", " +awd
                    
                    
                    
    ol_year = 0
    pr_year = 0
    pub_year = 0
    if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
        ol_year = int(wk['published-online']['date-parts'][0][0])
    if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
        pr_year = int(wk['published-print']['date-parts'][0][0])
    if pr_year > 0 and ol_year > 0:
        if pr_year > ol_year:
            pub_year = ol_year
        else:
            pub_year = pr_year
    elif ol_year > 0:
        pub_year = ol_year
    elif pr_year > 0:
        pub_year = pr_year
        
    print(art_authors,"|",pub_year,"|",wk['title'][0],
          "|", wk['DOI'],"|", fund_award)     
    this_pub = {}
    this_pub['authors'] = art_authors
    this_pub['year'] = pub_year
    this_pub['title'] = wk['title'][0]
    this_pub['DOI'] = wk['DOI']
    if not wk['DOI'] in foud_pubs:
         foud_pubs[wk['DOI']]= this_pub
    

Stößer, Tim, Williams, Charlotte K. | 2018 | Selective Polymerization Catalysis from Monomer Mixtures: Using a Commercial Cr‐Salen Catalyst To Access ABA Block Polyesters | 10.1002/anie.201801400 | EP/K014668/1
Stößer, Tim, Williams, Charlotte K. | 2018 | Selektive Katalyse für Polymerisationen mit Monomermischungen: Verwendung eines kommerziellen Cr‐Salen‐Katalysators zur Herstellung von ABA‐Blockpolyestern | 10.1002/ange.201801400 | EP/K014668/1
Plajer, Alex J., Williams, Charlotte K. | 2021 | Heterotrimetallic Carbon Dioxide Copolymerization and Switchable Catalysts: Sodium is the Key to High Activity and Unusual Selectivity | 10.1002/anie.202101180 | EP/R027129/1
Plajer, Alex J., Williams, Charlotte K. | 2021 | Heterotrimetallic Carbon Dioxide Copolymerization and Switchable Catalysts: Sodium is the Key to High Activity and Unusual Selectivity | 10.1002/ange.202101180 | EP/R027129/1
Stößer, Tim, Mulryan, Daniel, Williams, Charlotte K. | 2018 | Switch Catalysis To Deliver Multi‐Bloc

In [ ]:

aw = works.query(author="Williams").filter(has_funder='true').filter(from_pub_date='2020')

i = 0
ukch_wks =[] 
for wk in aw:
    awd_list = []    
    for fdr in wk['funder']:
        #if 'DOI' in fdr.keys() and fdr['DOI'] == '10.13039/501100000266':
         if 'award' in fdr.keys():
            awds = 0
            for awd in fdr['award']:
                if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                           'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                    awd_list.append(awd)
                        
    if len(awd_list) > 0:
        i+=1
        print(i, "* funder:", wk['funder'])
        print("* DOI", wk['DOI'])
        print("* Author:", wk['author'])
        print(awd_list)
        ukch_wks.append(wk)
        has_funder = True

len(ukch_wks) 

In [105]:
works = Works()
# get all the documents from 2012 which have a funder from 2012 and see if they list ukch grant numbers
pub_w_grant = works.filter(has_funder='true').filter(from_online_pub_date='2012')

ukch_grant = award_in_crossref(pub_w_grant)


In [117]:
foud_pubs = {}
for wk in ukch_grant:
    art_authors = ""
    if 'author' in wk.keys() :
        for autr in wk['author']:
            if art_authors == "":
                art_authors = autr['family']+", " + (", "+ autr ['given'] if 'given' in autr.keys() else "" )
            else:
                art_authors += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in autr.keys() else "" )
    fund_award = ""
    for fdr in wk['funder']:
        if 'award' in fdr.keys():
          for awd in fdr['award']:
               if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                          'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                    if fund_award  == "":
                        fund_award = awd
                    else:
                        fund_award += ", " +awd
                    
    ol_year = 0
    pr_year = 0
    pub_year = 0
    if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
        ol_year = int(wk['published-online']['date-parts'][0][0])
    if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
        pr_year = int(wk['published-print']['date-parts'][0][0])
    if pr_year > 0 and ol_year > 0:
        if pr_year > ol_year:
            pub_year = ol_year
        else:
            pub_year = pr_year
    elif ol_year > 0:
        pub_year = ol_year
    elif pr_year > 0:
        pub_year = pr_year
        
    print(art_authors,"|",pub_year,"|",wk['title'][0],
          "|", wk['DOI'],"|", fund_award)     
    this_pub = {}
    this_pub['authors'] = art_authors
    this_pub['year'] = pub_year
    this_pub['title'] = wk['title'][0]
    this_pub['DOI'] = wk['DOI']
    if not wk['DOI'] in foud_pubs:
         foud_pubs[wk['DOI']]= this_pub

if len(foud_pubs) > 0:
    csv_rw.write_csv_data(foud_pubs, 'cr_check_202110a.csv') 

Plajer, , Alex J., Williams, Charlotte K. | 2021 | Heterotrimetallic Carbon Dioxide Copolymerization and Switchable Catalysts: Sodium is the Key to High Activity and Unusual Selectivity | 10.1002/ange.202101180 | EP/R027129/1


In [112]:
this_pub

{'authors': 'Plajer, , Alex J., Williams, Charlotte K.',
 'year': 2021,
 'title': 'Heterotrimetallic Carbon Dioxide Copolymerization and Switchable Catalysts: Sodium is the Key to High Activity and Unusual Selectivity',
 'DOI': '10.1002/ange.202101180'}

In [3]:
# managing files and file paths
from pathlib import Path

# library for connecting to the db
import lib.handle_db as dbh

# import custom functions (common to various notebooks)
import processing_functions as pr_fs


In [57]:
# get the authors list from DB and search on CR for papers

i_idx = 1
these_dois = {}
#2 currend app DB
ukchapp_db = "db_files/app_db20210702.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()

author_list=pr_fs.get_authors_list(ukchapp_db)
print(author_list)

ukch_grant = []
for auth in author_list:
    works = Works()
    # start looking-up from 2018
    author_wks = works.query(author=auth[0]).filter(has_funder='true').filter(from_pub_date='2018')
    ukch_grant += award_in_crossref(author_wks)

foud_pubs = {}
for wk in ukch_grant:
    art_authors = ""
    if 'author' in wk.keys() :
        for autr in wk['author']:
            if art_authors == "":
                art_authors = autr['family']+", " + (", "+ autr ['given'] if 'given' in authr.keys() else "" )
            else:
                art_authors += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in authr.keys() else "" )
    fund_award = ""
    for fdr in wk['funder']:
        if 'award' in fdr.keys():
          for awd in fdr['award']:
               if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                          'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                    if fund_award  == "":
                        fund_award = awd
                    else:
                        fund_award += ", " +awd
                    
    ol_year = 0
    pr_year = 0
    pub_year = 0
    if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
        ol_year = int(wk['published-online']['date-parts'][0][0])
    if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
        pr_year = int(wk['published-print']['date-parts'][0][0])
    if pr_year > 0 and ol_year > 0:
        if pr_year > ol_year:
            pub_year = ol_year
        else:
            pub_year = pr_year
    elif ol_year > 0:
        pub_year = ol_year
    elif pr_year > 0:
        pub_year = pr_year
        
    print(art_authors,"|",pub_year,"|",wk['title'][0],
          "|", wk['DOI'],"|", fund_award)     
    this_pub = {}
    this_pub['authors'] = art_authors
    this_pub['year'] = pub_year
    this_pub['title'] = wk['title'][0]
    this_pub['DOI'] = wk['DOI']
    if not wk['DOI'] in foud_pubs:
         foud_pubs[wk['DOI']]= this_pub

[('Gibson', 'Emma K.'), ('Morgan', 'David J.'), ('Dimitratos', 'Nikolaos'), ('Kiely', 'Christopher J.'), ('Beale', 'Andrew M.'), ('Hutchings', 'Graham J.'), ('Wells', 'Peter P.'), ('Catlow', 'C. Richard A.'), ('Matam', 'Santhosh K.'), ('Greenaway', 'Alex G.'), ('Raja', 'Robert'), ('Collier', 'Paul'), ('Chutia', 'Arunabhiram'), ('Hardacre', 'Christopher'), ('Rogers', 'Scott M.'), ('Lennon', 'David'), ('Parker', 'Stewart F.'), ('Lezcano-González', 'Inés'), ("O'Malley", 'Alexander J.'), ('Silverwood', 'Ian P.'), ('Callison', 'June'), ('Kondrat', 'Simon A.'), ('Decarolis', 'Donato'), ('Celorrio', 'Verónica'), ('Fermín', 'David J.'), ('Bowker', 'Michael'), ('Jones', 'Wilm'), ('Taylor', 'Stuart H.'), ('Xu', 'Shaojun')]
Jones, Wilm, Wells, Peter. P., Gibson, Emma K., Chutia, Arunabhiram, Silverwood, Ian P., Catlow, C. Richard A., Bowker, Michael | 2019 | Carbidisation of Pd Nanoparticles by Ethene Decomposition with Methane Production | 10.1002/cctc.201900795 | EP/K014714/1
Bartlett, Stuart A

KeyError: 'given'

In [1]:
foud_pubs = {}
for wk in ukch_grant:
    art_authors = ""
    if 'author' in wk.keys() :
        for autr in wk['author']:
            if art_authors == "":
                art_authors = autr['family']+", " + (", "+ autr ['given'] if 'given' in authr.keys() else "" )
            else:
                art_authors += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in autr.keys() else "" )
    fund_award = ""
    for fdr in wk['funder']:
        if 'award' in fdr.keys():
          for awd in fdr['award']:
               if awd in ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
                          'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1']:
                    if fund_award  == "":
                        fund_award = awd
                    else:
                        fund_award += ", " +awd
                    
    ol_year = 0
    pr_year = 0
    pub_year = 0
    if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
        ol_year = int(wk['published-online']['date-parts'][0][0])
    if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
        pr_year = int(wk['published-print']['date-parts'][0][0])
    if pr_year > 0 and ol_year > 0:
        if pr_year > ol_year:
            pub_year = ol_year
        else:
            pub_year = pr_year
    elif ol_year > 0:
        pub_year = ol_year
    elif pr_year > 0:
        pub_year = pr_year
        
    print(art_authors,"|",pub_year,"|",wk['title'][0],
          "|", wk['DOI'],"|", fund_award)     
    this_pub = {}
    this_pub['authors'] = art_authors
    this_pub['year'] = pub_year
    this_pub['title'] = wk['title'][0]
    this_pub['DOI'] = wk['DOI']
    if not wk['DOI'] in foud_pubs:
         foud_pubs[wk['DOI']]= this_pub

NameError: name 'ukch_grant' is not defined

In [4]:
# Connecting to the db
import lib.handle_db as dbh

# read and write csv files
import lib.handle_csv as csv_rw


In [13]:
#read csv results from CR search

input_file = './CR_search/cr_search202110b.csv'
id_field = 'num'
cr_results, headings = csv_rw.get_csv_data(input_file, id_field)

print(cr_results)
# get the publications list from the app database
ukchapp_db = "./db_files/app_db20211005.sqlite3"
db_pubs = pr_fs.get_pub_app_data(ukchapp_db)

# collect cr dois
check_dois = []
for entry in cr_results:
    if not cr_results[entry]['doi'].strip() in check_dois:
        check_dois.append(cr_results[entry]['doi'].strip())

print(check_dois)
        
# get all dois in DB
db_dois = [db_pubs[i][2] for i in range(0,len(db_pubs))]

print(db_dois)
# get all dois in CR which are not in the DB

cr_not_in_db = [x for x in check_dois if x not in db_dois]

print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print(cr_not_in_db)
print(len(cr_not_in_db))
out_cr_check={}
# collect cr dois
for entry in cr_results:
    if cr_results[entry]['doi'].strip() in cr_not_in_db:
        if out_cr_check=={}:
            out_cr_check[entry] = cr_results[entry]
        elif cr_results[entry]['doi'] not in [out_cr_check[list(out_cr_check.keys())[i]]['doi'] for i in range(0,len(out_cr_check))]:
            out_cr_check[entry] = cr_results[entry]

print(out_cr_check)
if len(cr_not_in_db) > 0:
    csv_rw.write_csv_data(out_cr_check, 'cr_check_202110b.csv')           

{1: OrderedDict([('num', '1'), ('authors', 'Doble, Megan V., Ward, Andrew C.C., Deuss, Peter J., Jarvis, Amanda G., Kamer, Paul C.J.'), ('year', '2014'), ('title', 'Catalyst design in oxidation chemistry; from KMnO4 to artificial metalloenzymes'), ('doi', '10.1016/j.bmc.2014.07.002'), ('award', 'EP/K014706/1, EP/K014668/1, EP/K014854/1, EP/K014714/1')]), 2: OrderedDict([('num', '2'), ('authors', 'Hinde, Christopher S., Gill, Arran M., Wells, Peter P., Hor, T. S. Andy, Raja, Robert'), ('year', '2015'), ('title', 'Utilizing Benign Oxidants for Selective Aerobic Oxidations Using Heterogenized Platinum Nanoparticle Catalysts'), ('doi', '10.1002/cplu.201500195'), ('award', 'SP8071-6, EP/K014706/1, EP/K014668/1, EP/K014854/1, EP/K014714/1, R143-000-492-592')]), 3: OrderedDict([('num', '3'), ('authors', 'Dzielendziak, Agnieszka S., Blake, James I. R., Bounds, Richard, Wilkinson, Karl A., Carravetta, Marina, Chambers, Alan R., Skylaris, Chris-Kriton, Raja, Robert'), ('year', '2015'), ('title',